### multiple subdomain 결과 재정렬

In [ ]:
# import json
# domain = '경제'
# with open(f'evaluation/eval_data/multiple_subdomain_results/{domain}_subdomain_classified.json', 'r', encoding='utf-8') as f:
    # data = json.load(f)

#  new_data = []
# for d in data:
#     if d['qna_subdomain_reason'] in d['qna_reason']:
#         d['qna_reason'] = d['qna_reason'].replace(d['qna_subdomain_reason'], '').strip()
#         # print(d)
#     new_d = {
#         'file_id': d['file_id'],
#         'title': d['title'],
#         'chapter': d['chapter'],
#         'qna_id': d['qna_id'],
#         'qna_domain': d['qna_domain'],
#         'qna_subdomain': d['qna_subdomain'],
#         'qna_domain_reason': d['qna_reason'],
#         'qna_subdomain_reason': d['qna_subdomain_reason'],
#         'qna_question': d['qna_question'],
#         'qna_options': d['qna_options'],
#         'qna_answer': d['qna_answer'],
#         'qna_explanation': d['qna_explanation']
#     }
#     new_data.append(new_d)
    
# with open(f'evaluation/eval_data/subdomain_results_old/{domain}_subdomain_classified.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

### multiple options 오류들

In [31]:
import json
import re

domain = '경제' 
with open(f'evaluation/eval_data/multiple_subdomain_results/{domain}_subdomain_classified.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# ①②③④⑤로 시작하지 않는 경우를 찾기
invalid_options = []

for d in data:
    if d['qna_options'] and isinstance(d['qna_options'], list):
        options = d['qna_options']
        
        # 각 옵션을 확인
        for i, option in enumerate(options):
            # ①②③④⑤로 시작하지 않는 경우 찾기
            if not re.match(r'^[①②③④⑤]', option.strip()):
                invalid_options.append({
                    'file_id': d['file_id'],
                    'qna_id': d['qna_id'],
                    'option_index': i,
                    'original_option': option,
                    # 'first_char': option.strip()[0] if option.strip() else ''
                })

print(f"①②③④⑤로 시작하지 않는 options 수: {len(invalid_options)}")

invalid_options

①②③④⑤로 시작하지 않는 options 수: 716


[{'file_id': 'SS0302',
  'qna_id': 'q_0138_0003',
  'option_index': 4,
  'original_option': '구분 채권 주식'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0139_0001',
  'option_index': 4,
  'original_option': '분류 기준 채권의 종류'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0142_0001',
  'option_index': 4,
  'original_option': '㉠ ㉡'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0142_0001',
  'option_index': 5,
  'original_option': '㉠ ㉡'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0143_0001',
  'option_index': 4,
  'original_option': '수익률 수익률의 종류'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0143_0001',
  'option_index': 5,
  'original_option': '수익률 수익률의 종류'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0156_0002',
  'option_index': 4,
  'original_option': '㉠ ㉡ ㉢'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0202_0003',
  'option_index': 4,
  'original_option': '㉠ ㉡'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0202_0003',
  'option_index': 5,
  'original_option': '㉠ ㉡'},
 {'file_id': 'SS0302',
  'qna_id': 'q_0421_0006',
  'option_index': 

## 모의고사 만들기 파이프라인

In [59]:
import os, json, random

BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

with open(os.path.join(BASE_DIR, 'classification_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)


for json_file in os.listdir(BASE_DIR):
    if not json_file.endswith("_subdomain_classified.json"):
        continue
    
    # if json_file != '세무_subdomain_classified.json':
        # continue
    
    with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    domain = json_file.replace("_subdomain_classified.json", "")
    print(f"=====================================\n<<< {domain} 분야 모의고사용 문제 추출 >>>")
    
    total_questions = len(data)
    if total_questions == stats[domain]['total_questions']:
        mock_data = []
        need_more = {}
        mock_total_questions = round(total_questions * 0.3)
        print(f"원본 문제 수: {total_questions}")
        print(f"추출 대상 문제 수: {mock_total_questions}")

        # subdomain 별로 모의고사 만들기
        for subdomain in stats[domain]['subdomain_distribution'].keys():
            subdomain_data = [d for d in data if d['qna_subdomain'] == subdomain]
            try:
                mock_subdomain_data = random.sample(subdomain_data, mock_total_questions // len(stats[domain]['subdomain_distribution']))
            except ValueError:
                mock_subdomain_data = subdomain_data
                need_more[subdomain] = mock_total_questions // len(stats[domain]['subdomain_distribution']) - len(mock_subdomain_data)
            
            print(f"  - {subdomain}: {len(mock_subdomain_data)}")
            mock_data.extend(mock_subdomain_data)

        print(f" ====> 총 추출 문제: {len(mock_data)}")
        if need_more:
            print(f" ====> 모자란 문제: {need_more}")
    else:
        print("갯수가 안맞아!")
    
    with open(f'{BASE_DIR}/mock_data/{domain}_mock_data.json', 'w', encoding='utf-8') as f:
        json.dump(mock_data, f, ensure_ascii=False, indent=4)


<<< 경제 분야 모의고사용 문제 추출 >>>
원본 문제 수: 10241
추출 대상 문제 수: 3072
  - 미시경제학: 768
  - 경제정책 및 시사경제: 666
  - 거시경제학: 768
  - 국제경제학: 768
 ====> 총 추출 문제: 2970
 ====> 모자란 문제: {'경제정책 및 시사경제': 102}
<<< 보상처리 분야 모의고사용 문제 추출 >>>
원본 문제 수: 2271
추출 대상 문제 수: 681
  - 보험금심사: 227
  - 손해사정실무: 227
  - 분쟁조정 및 금융소비자보호: 227
 ====> 총 추출 문제: 681
<<< 통계 분야 모의고사용 문제 추출 >>>
원본 문제 수: 2517
추출 대상 문제 수: 755
  - 기초통계학: 188
  - 응용통계 및 시계열분석: 188
  - 추론통계학: 188
  - 데이터조사 및 분석활용: 188
 ====> 총 추출 문제: 752
<<< 리스크관리 분야 모의고사용 문제 추출 >>>
원본 문제 수: 4637
추출 대상 문제 수: 1391
  - 리스크기초이론: 347
  - 보험리스크관리: 113
  - 금융리스크측정기법: 347
  - 통합리스크관리: 228
 ====> 총 추출 문제: 1035
 ====> 모자란 문제: {'보험리스크관리': 234, '통합리스크관리': 119}
<<< 보험계약 분야 모의고사용 문제 추출 >>>
원본 문제 수: 5231
추출 대상 문제 수: 1569
  - 보험상품구조: 392
  - 언더라이팅 및 재보험: 392
  - 보험법 및 계약이론: 392
  - 보험회계 및 공시: 117
 ====> 총 추출 문제: 1293
 ====> 모자란 문제: {'보험회계 및 공시': 275}
<<< 회계 분야 모의고사용 문제 추출 >>>
원본 문제 수: 6409
추출 대상 문제 수: 1923
  - 재무회계: 641
  - 회계기준 및 감사: 410
  - 관리회계: 641
 ====> 총 추출 문제: 1692
 ====> 모자란 문제: {'회계기준 

### 수정한 도메인 원본에 반영하기?

### 분류 조정하기

In [55]:
import os, json

BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

for json_file in os.listdir(BASE_DIR):
    if not json_file.endswith("_subdomain_classified.json"):
        continue
    
    with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    domain = json_file.replace("_subdomain_classified.json", "")
    
    new_data = []
    other_domain = {}
    for d in data:
        if d['qna_domain'] != domain:
            other_domain[d['qna_domain']] = d
        else:
            new_data.append(d)
    
    with open(os.path.join(BASE_DIR, f'{domain}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
        json.dump(new_data, f, ensure_ascii=False, indent=4)
    
    for other_domain_key in other_domain.keys():
        with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'r', encoding='utf-8') as f:
            other_domain_data = json.load(f)
        
        other_domain_data.append(other_domain[other_domain_key])
                
        with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
            json.dump(other_domain_data, f, ensure_ascii=False, indent=4)
